<a href="https://colab.research.google.com/github/ashavish/Hackathon/blob/master/AV_NLP_HACK/MultiLabelBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data - train, test and tags

In [38]:
from google.colab import files
files.upload()

In [10]:
# !unzip -q Test_H6bikL1.zip
# !unzip -q Train_aO7sTW8.zip
!ls

__MACOSX     Tags.csv  Test_H6bikL1.zip   Train.csv
sample_data  Test.csv  Train_aO7sTW8.zip


In [6]:
#!pip install transformers

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from transformers import BertTokenizer, BertModel, BertConfig
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Creating Data Loaders

In [24]:
class CustomDataset(Dataset):

    def __init__(self, text_input,target_labels, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.comment_text = text_input
        self.targets = target_labels
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }


class TestDataset(Dataset):

    def __init__(self, text_input,tokenizer, max_len):
        self.tokenizer = tokenizer
        self.comment_text = text_input
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }

In [25]:
import pandas as pd
import numpy as np
import string

train_data = pd.read_csv("Train.csv")
tags = pd.read_csv("Tags.csv")
test_data = pd.read_csv("Test.csv")

TOPIC_COLS = ['Computer Science','Mathematics','Physics','Statistics']
TAGS = np.array(list(tags['Tags']))

train_data['ABSTRACT'] = train_data['ABSTRACT'].str.lower()
test_data['ABSTRACT'] = test_data['ABSTRACT'].str.lower()


train_data['ABSTRACT'] = train_data['ABSTRACT'].str.translate(str.maketrans('', '', string.punctuation))
test_data['ABSTRACT'] = test_data['ABSTRACT'].str.translate(str.maketrans('', '', string.punctuation))


abstracts = list(train_data['ABSTRACT'])
labels = np.array(train_data[TAGS])


In [26]:
# Config parameters

MAX_LEN = 300
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 4
LEARNING_RATE = 3e-05

In [10]:

train_sents, val_sents,train_labels,val_labels = train_test_split(abstracts,labels,test_size=0.2,random_state=2)


print("Train Dataset: {}".format(len(train_sents)))
print("Val Dataset: {}".format(len(val_sents)))

test_sents = list(test_data['ABSTRACT'])

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
                  
training_set = CustomDataset(train_sents,train_labels, tokenizer, MAX_LEN)
validation_set = CustomDataset(val_sents,val_labels, tokenizer, MAX_LEN)

testing_set = TestDataset(test_sents,tokenizer, MAX_LEN)

Train Dataset: 11203
Val Dataset: 2801


In [11]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **val_params)
testing_loader = DataLoader(testing_set, **test_params)

# Defining the model - Pretrained BERT Model + Dropout + Linear Layer With Sigmoid activation

In [37]:
import transformers
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 25)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [38]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

# Model training as per config params

In [39]:
def train(epoch):
    model.train()
    # all_outputs = []
    # all_targets = []
    for batch_no,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if batch_no%500==0:
            print(f'Epoch: {epoch},Loss:  {loss.item()}')        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #     all_targets.extend(targets.cpu().detach().numpy().tolist())
    #     all_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist()) 
    # all_outputs = np.array(all_outputs) >= 0.5
    # accuracy = metrics.accuracy_score(all_targets, all_outputs)
    # f1_score_micro = metrics.f1_score(all_targets, all_outputs, average='micro')
    # f1_score_macro = metrics.f1_score(all_targets, all_outputs, average='macro')
    # print(f"Training Accuracy Score = {accuracy}")
    # print(f"Training F1 Score (Micro) = {f1_score_micro}")
    # print(f"Training F1 Score (Macro) = {f1_score_macro}")

In [40]:
def validate_model(data_loader,target=True):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(data_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            if target:
              targets = data['targets'].to(device, dtype = torch.long)
              fin_targets.extend(targets.cpu().detach().numpy().tolist())                     
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets
    
def validation_metrics(epoch):
    outputs, targets = validate_model(validation_loader)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Validation Accuracy Score = {accuracy}")
    print(f"Validation F1 Score (Micro) = {f1_score_micro}")
    print(f"Validation F1 Score (Macro) = {f1_score_macro}")

In [41]:
for epoch in range(EPOCHS):
  train(epoch)
  validation_metrics(epoch)


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0,Loss:  0.6837453842163086
Epoch: 0,Loss:  0.14390690624713898
Epoch: 0,Loss:  0.12771940231323242
Validation Accuracy Score = 0.349875044626919
Validation F1 Score (Micro) = 0.5738607911867801
Validation F1 Score (Macro) = 0.4684526679356458
Epoch: 1,Loss:  0.10783501714468002
Epoch: 1,Loss:  0.08357148617506027
Epoch: 1,Loss:  0.07525722682476044
Validation Accuracy Score = 0.47197429489468046
Validation F1 Score (Micro) = 0.6626524160770736
Validation F1 Score (Macro) = 0.6401829314015748
Epoch: 2,Loss:  0.03012237884104252
Epoch: 2,Loss:  0.07207632064819336
Epoch: 2,Loss:  0.10212679952383041
Validation Accuracy Score = 0.4919671545876473
Validation F1 Score (Micro) = 0.6828771112405359
Validation F1 Score (Macro) = 0.6672225933781643
Epoch: 3,Loss:  0.04248304292559624
Epoch: 3,Loss:  0.0525391660630703
Epoch: 3,Loss:  0.09547990560531616
Validation Accuracy Score = 0.4791146019278829
Validation F1 Score (Micro) = 0.6907730673316709
Validation F1 Score (Macro) = 0.6894422

In [42]:
# model.save_pretrained("bert_finetuned_multilabel")

output_model_file = 'pytorch_bertnofreeze_multilabel.bin'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_pretrained("bert_tok_multilabel")

('bert_tok_multilabel/tokenizer_config.json',
 'bert_tok_multilabel/special_tokens_map.json',
 'bert_tok_multilabel/vocab.txt',
 'bert_tok_multilabel/added_tokens.json')

In [ ]:
model = torch.load("pytorch_bertnofreeze_multilabel.bin")
tokenizer = BertTokenizer.from_pretrained("bert_tok_multilabel")

# Get best thresholds for different labels

In [36]:
from sklearn.metrics import f1_score

# Get best threshold for each label
def get_cut_offthreshold(y_pred_prob,validation_set,TAGS):
    '''
    Get best threshold cut off for different labels to maximize the micro F1 score
    '''
    thresholds = np.array(list(range(0,100)))/100.0
    best_thresholds = []
    for idx in range(0,25):
        scores = [f1_score(validation_set[:,idx], y_pred_prob[:,idx] > thresh, average='micro') for thresh in thresholds]
        best_thresh = thresholds[np.argmax(scores)]
        best_thresholds.append(best_thresh)
    return best_thresholds

def get_predictions(pred_prob,best_thresholds,TAGS):
    '''
    Get predictions based on probabilities and class specific thresholds
    '''
    predictions = np.zeros((pred_prob.shape[0],len(TAGS)))
    for idx in range(0,25):
        predictions[:,idx] = pred_prob[:,idx] > best_thresholds[idx]    
    return predictions

In [52]:
fin_outputs,fin_targets = validate_model(validation_loader)

y_pred_prob = np.array(fin_outputs)

best_thresholds = get_cut_offthreshold(y_pred_prob,val_labels,TAGS)

y_pred = get_predictions(y_pred_prob,best_thresholds,TAGS)

print("F1 Score on Validation Set", f1_score(val_labels, y_pred, average='micro'))

F1 Score on Validation Set 0.7132031477703293


In [53]:
import numpy as np
np.save("best_thresholds.npy",best_thresholds)


# Predicting on Test set

In [54]:
# Predicting on Test

fin_outputs,fin_targets = validate_model(testing_loader,target=False)

y_pred_test_prob = np.array(fin_outputs)

predictions = get_predictions(y_pred_test_prob,best_thresholds,TAGS)

result = pd.DataFrame(predictions)
result.columns = TAGS
result['id'] = test_data['id']
result.to_csv("bert_finetuned.csv",index=False)


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [55]:
!ls

 bert_finetuned.csv		       Tags.csv
 bert_tok_multilabel		       Test.csv
 best_thresholds.npy		      'Test_H6bikL1 (1).zip'
 __MACOSX			       Test_H6bikL1.zip
 pytorch_bertnofreeze_multilabel.bin   Train_aO7sTW8.zip
 sample_data			       Train.csv


In [56]:
# files.download("pytorch_bertnofreeze_multilabel.bin")
files.download("bert_finetuned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Experiment: BERT Model finetuning with Topic Columns

In [11]:
class CustomDataset(Dataset):

    def __init__(self, text_input,target_labels,topic_cols, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.comment_text = text_input
        self.targets = target_labels
        self.max_len = max_len
        self.topic_cols = topic_cols

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float),
            'topic_cols':torch.tensor(self.topic_cols[index], dtype=torch.long)
        }


class TestDataset(Dataset):

    def __init__(self, text_input,topic_cols,tokenizer, max_len):
        self.tokenizer = tokenizer
        self.comment_text = text_input
        self.max_len = max_len
        self.topic_cols = topic_cols

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'topic_cols':torch.tensor(self.topic_cols[index], dtype=torch.long)
        }

In [14]:
import pandas as pd
import numpy as np
import string

train_data = pd.read_csv("Train.csv")
tags = pd.read_csv("Tags.csv")
test_data = pd.read_csv("Test.csv")

TOPIC_COLS = ['Computer Science','Mathematics','Physics','Statistics']
TAGS = np.array(list(tags['Tags']))

train_data['ABSTRACT'] = train_data['ABSTRACT'].str.lower()
test_data['ABSTRACT'] = test_data['ABSTRACT'].str.lower()


train_data['ABSTRACT'] = train_data['ABSTRACT'].str.translate(str.maketrans('', '', string.punctuation))
test_data['ABSTRACT'] = test_data['ABSTRACT'].str.translate(str.maketrans('', '', string.punctuation))


abstracts = list(train_data['ABSTRACT'])
labels = np.array(train_data[TAGS])
topic_cols = np.array(train_data[TOPIC_COLS])


In [17]:
# Config parameters

MAX_LEN = 300
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 4
LEARNING_RATE = 3e-05

In [18]:
train_sents, val_sents,train_labels,val_labels,train_topic_cols,val_topic_cols = train_test_split(abstracts,labels,topic_cols,test_size=0.2,random_state=2)
print("Train Dataset: {}".format(len(train_sents)))
print("Val Dataset: {}".format(len(val_sents)))

test_sents = list(test_data['ABSTRACT'])
test_topic_cols = np.array(test_data[TOPIC_COLS])

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

training_set = CustomDataset(train_sents,train_labels,train_topic_cols, tokenizer, MAX_LEN)
validation_set = CustomDataset(val_sents,val_labels,val_topic_cols,tokenizer, MAX_LEN)

testing_set = TestDataset(test_sents,test_topic_cols,tokenizer, MAX_LEN)

Train Dataset: 11203
Val Dataset: 2801


In [19]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **val_params)
testing_loader = DataLoader(testing_set, **test_params)

In [20]:
torch.cuda.empty_cache() 

In [27]:
import transformers
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.tc = torch.nn.Linear(4,4)        
        self.l2 = torch.nn.Dropout(0.3)        
        self.l3 = torch.nn.Linear(768+4, 25)
    
    def forward(self, ids, mask, token_type_ids,topic_inputs):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output_tc = self.tc(topic_inputs)
        output_2 = torch.cat((output_2,output_tc),dim=1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [28]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [32]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        topic_cols = data['topic_cols'].to(device, dtype = torch.float)
        outputs = model(ids, mask,token_type_ids,topic_cols)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [33]:
def validate_model(data_loader,target=True):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(data_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            topic_cols = data['topic_cols'].to(device, dtype = torch.float)
            outputs = model(ids, mask,token_type_ids,topic_cols)
            if target:
              targets = data['targets'].to(device, dtype = torch.long)
              fin_targets.extend(targets.cpu().detach().numpy().tolist())                     
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets
    
def validation_metrics(epoch):
    outputs, targets = validate_model(validation_loader)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Validation Accuracy Score = {accuracy}")
    print(f"Validation F1 Score (Micro) = {f1_score_micro}")
    print(f"Validation F1 Score (Macro) = {f1_score_macro}")

In [34]:
for epoch in range(EPOCHS):
  train(epoch)
  validation_metrics(epoch)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  0.7956587076187134
Epoch: 0, Loss:  0.14777939021587372
Epoch: 0, Loss:  0.08795464038848877
Validation Accuracy Score = 0.3495180292752588
Validation F1 Score (Micro) = 0.5636424061149883
Validation F1 Score (Macro) = 0.49224254006674156
Epoch: 1, Loss:  0.08724609762430191
Epoch: 1, Loss:  0.11556632816791534
Epoch: 1, Loss:  0.06546955555677414
Validation Accuracy Score = 0.4759014637629418
Validation F1 Score (Micro) = 0.6958206476612234
Validation F1 Score (Macro) = 0.6821884813610378
Epoch: 2, Loss:  0.07216538488864899
Epoch: 2, Loss:  0.054021477699279785
Epoch: 2, Loss:  0.07967785745859146
Validation Accuracy Score = 0.47697250981792216
Validation F1 Score (Micro) = 0.683189349950432
Validation F1 Score (Macro) = 0.6764711076362524
Epoch: 3, Loss:  0.04558594152331352
Epoch: 3, Loss:  0.03672091290354729
Epoch: 3, Loss:  0.04236995428800583
Validation Accuracy Score = 0.4923241699393074
Validation F1 Score (Micro) = 0.7102983638113569
Validation F1 Score (Mac

In [37]:
fin_outputs,fin_targets = validate_model(validation_loader)

y_pred_prob = np.array(fin_outputs)

best_thresholds = get_cut_offthreshold(y_pred_prob,val_labels,TAGS)

y_pred = get_predictions(y_pred_prob,best_thresholds,TAGS)

print("F1 Score on Validation Set", f1_score(val_labels, y_pred, average='micro'))

F1 Score on Validation Set 0.7152028012839218


In [ ]:
# Predicting on Test

fin_outputs,fin_targets = validate_model(testing_loader,target=False)

y_pred_test_prob = np.array(fin_outputs)

predictions = get_predictions(y_pred_test_prob,best_thresholds,TAGS)

result = pd.DataFrame(predictions)
result.columns = TAGS
result['id'] = test_data['id']
result.to_csv("bert_finetuned_topic_cols.csv",index=False)